In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from multiproduct import generate_dataset

In [3]:
n_nodes, supplies, demands, costs, capacities = generate_dataset(
                nodes=10, levels=3, total_supplies=[500,400,300], 
                total_demands=[400,600,400], transp_costs=(10,100), 
                random_state=42)

In [4]:
from multiproduct import generate_ampl
generate_ampl(n_nodes, supplies, demands, costs, capacities,
              output_file='modelos/test10.mod')

In [49]:
n_nodes

[2, 3, 5]

In [5]:
supplies

[[200, 300, 0], [130, 270, 200], [170, 130, 100]]

In [6]:
capacities

[[500, 700], [437, 500, 619], [150, 270, 320, 230, 430]]

In [7]:
supplies

[[200, 300, 0], [130, 270, 200], [170, 130, 100]]

In [8]:
demands

[[10, 50, 90, 40, 210, 100],
 [110, 180, 100, 120, 90, 0],
 [30, 40, 130, 70, 130, 0]]

In [206]:
len(np.array(supplies).shape)

2

In [4]:
def transport(left, right, left_cap, right_cap, X, 
              item, j, k, demand_limit=None):
    '''
    left: [items, n_nodesL]
    right: [items, n_nodesR]
    left_cap: [items, n_nodesL] or [n_nodesL]
    right_cap: [items, n_nodesR]
    X: [items, n_nodesL, n_nodesR]
    demand_limit: [n_nodesR]
    '''
#     print('caps:', left_cap, right_cap)
    item_cap = left_cap[item][j] if len(np.array(left_cap).shape)==2 \
                                 else left_cap[j]
    assert item_cap >= left[item][j] 
    assert right_cap[item][k] >= right[item][k] 
    available = min(item_cap - left[item][j], 
                    right_cap[item][k] - right[item][k])
    if demand_limit is not None: 
        available = min(available, 
                        int(demand_limit[item][k] * right_cap[item][k]))
    left[item][j] += available
    right[item][k] += available
    X[item][j][k] += available
    return available

In [12]:
from itertools import product
list(product(range(3),range(3)))

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [13]:
ch = np.random.randint(0, 100, (3, 5))

In [14]:
ch

array([[76,  2, 69, 71, 26],
       [ 8, 61, 36, 96, 50],
       [43, 23, 78, 58, 31]])

In [24]:
for i,k in sorted(product(range(3), range(5)), key=lambda x: ch[x],
                reverse=True):
    print(i,k,ch[i][k])

1 3 96
2 2 78
0 0 76
0 3 71
0 2 69
1 1 61
2 3 58
1 4 50
2 0 43
1 2 36
2 4 31
0 4 26
2 1 23
1 0 8
0 1 2


In [5]:
def transportation_stage(I, J, K, n_nodes, left_cap, right_cap, X,
                         chrom, costs):
    '''
    stage: int [0..n-1)
    n_nodes: [levels]
    left_cap: [items, n_nodesL] or [n_nodesL]
    right_cap: [items, n_nodesR] (no dummies)
    costs: [n_nodesL, n_nodesR]
    '''
    left = np.zeros((I, J), dtype=int)
    right = np.zeros((I, K), dtype=int)
    chrom = np.reshape(chrom, (I, K))    
    for i,k in sorted(product(range(I), range(K)), 
                      key=lambda x: chrom[x],
                      reverse=True):
        for j in sorted(range(J), 
                      key=lambda x: costs[x,k]):
            items = transport(left, right, left_cap, right_cap, X, 
                              i, j, k)
#     print(left)
    return left

In [45]:
demands

[[10, 50, 90, 40, 210, 100],
 [110, 180, 100, 120, 90, 0],
 [30, 40, 130, 70, 130, 0]]

In [10]:
ch = np.random.permutation(n_nodes[-1] * n_items)

In [53]:
ch.reshape((3,5))

array([[ 5, 10,  4, 13,  9],
       [ 1, 11,  7,  0,  8],
       [12,  6,  3,  2, 14]])

In [111]:
X[-1]

array([[[  0.,   0.,   0.,  40.,   0.,   0.],
        [  0.,   0.,   0.,   0., 210.,   0.],
        [ 10.,  50.,  90.,   0.,   0.,   0.]],

       [[  0.,   0.,   0., 120.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  90.,   0.],
        [110., 180., 100.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,  70.,   0.,   0.],
        [  0.,   0.,   0.,   0., 130.,   0.],
        [ 30.,  40., 130.,   0.,   0.,   0.]]])

In [12]:
n_items = 3
X = [np.zeros((n_items,
               n_nodes[i] + int(i==0), 
               n_nodes[i+1] + int(i+1==len(n_nodes)-1))) \
     for i in range(len(n_nodes)-1)]

# stage = 1
# transportation_stage(stage, n_items, n_nodes, capacities[-2], 
#                      [x[:-1] for x in demands], X[stage],
#                      ch, costs[stage])

In [121]:
np.cumsum(n_nodes)

array([ 2,  5, 10])

In [137]:
chrom_idxs = [0] + list(np.cumsum([n_items * x for x in n_nodes][1:]))
chrom_stages = [(chrom_idxs[i],chrom_idxs[i+1]) for i in range(len(n_nodes)-1)]

In [138]:
n_nodes

[2, 3, 5]

In [139]:
chrom_stages

[(0, 9), (9, 24)]

In [6]:
def transportation_tree(chromosome, n_nodes, supplies, demands, costs, capacities):
    n_items = len(demands)
    X = [np.zeros((n_items,
               n_nodes[i] + int(i==0), 
               n_nodes[i+1] + int(i+1==len(n_nodes)-1))) \
         for i in range(len(n_nodes)-1)]
    chrom_idxs = [0] + list(np.cumsum([n_items * x for x in n_nodes][1:]))
    chrom_stages = [chromosome[chrom_idxs[i]:chrom_idxs[i+1]] \
                    for i in range(len(n_nodes)-1)]
    # Dummy demand
    right_cap = [x[:-1] for x in demands]
    right_cap = dummy_transportation(n_items, right_cap, X)
    for i in range(len(n_nodes)-2):
        stage = len(n_nodes) - i - 2
        right_cap = transportation_stage(n_items, n_nodes[stage],
                            n_nodes[stage+1], n_nodes, 
                            capacities[stage], right_cap, X[stage],
                            chrom_stages[stage], costs[stage])
    # Supply
    right_cap = transportation_stage(n_items, n_nodes[0],
                            n_nodes[1], n_nodes,
                            supplies, right_cap, X[0],
                            chrom_stages[0], costs[0])
    # Dummy supply
    return X

In [202]:
capacities[1]

[437, 500, 619]

In [203]:
capacities[0]

[500, 700]

In [201]:
supplies

[[200, 300, 0], [130, 270, 200], [170, 130, 100]]

In [17]:
np.random.seed(42)
chromosome = np.random.permutation(sum([n_items * sum(n_nodes[1:])]))

In [18]:
chromosome

array([ 8, 16,  0, 18, 11,  9, 13,  1, 21,  5,  2, 12, 15,  3,  4, 22, 17,
       20, 23,  7, 10, 14, 19,  6])

In [167]:
n_nodes

[2, 3, 5]

In [20]:
X = transportation_tree(chromosome, n_nodes, supplies, demands, 
                        costs, capacities)

In [214]:
X

[array([[[ 40.,   0., 150.],
         [  0., 210.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0., 130.],
         [  0.,  10., 260.],
         [  0.,   0.,   0.]],
 
        [[ 70.,   0., 100.],
         [  0., 130.,   0.],
         [  0.,   0.,   0.]]]), array([[[  0.,   0.,   0.,  40.,   0.,   0.],
         [  0.,   0.,   0.,   0., 210.,   0.],
         [ 10.,  50.,  90.,   0.,   0.,   0.]],
 
        [[  0.,   0.,   0., 120.,   0.,   0.],
         [  0.,   0.,   0.,   0.,  90.,   0.],
         [110., 180., 100.,   0.,   0.,   0.]],
 
        [[  0.,   0.,   0.,  70.,   0.,   0.],
         [  0.,   0.,   0.,   0., 130.,   0.],
         [ 30.,  40., 130.,   0.,   0.,   0.]]])]

In [228]:
demands

[[10, 50, 90, 40, 210, 100],
 [110, 180, 100, 120, 90, 0],
 [30, 40, 130, 70, 130, 0]]

In [230]:
X

[array([[[ 40.,   0., 150.],
         [  0., 210.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0., 130.],
         [  0.,  10., 260.],
         [  0.,   0.,   0.]],
 
        [[ 70.,   0., 100.],
         [  0., 130.,   0.],
         [  0.,   0.,   0.]]]), array([[[  0.,   0.,   0.,  40.,   0.,   0.],
         [  0.,   0.,   0.,   0., 210.,   0.],
         [ 10.,  50.,  90.,   0.,   0.,   0.]],
 
        [[  0.,   0.,   0., 120.,   0.,   0.],
         [  0.,   0.,   0.,   0.,  90.,   0.],
         [110., 180., 100.,   0.,   0.,   0.]],
 
        [[  0.,   0.,   0.,  70.,   0.,   0.],
         [  0.,   0.,   0.,   0., 130.,   0.],
         [ 30.,  40., 130.,   0.,   0.,   0.]]])]

In [192]:
X[0]

array([[[ 40.,   0., 150.],
        [  0., 210.,   0.],
        [  0.,   0.,   0.]],

       [[  0.,   0., 130.],
        [  0.,  10., 260.],
        [  0.,   0.,   0.]],

       [[ 70.,   0., 100.],
        [  0., 130.,   0.],
        [  0.,   0.,   0.]]])

In [193]:
X[1]

array([[[  0.,   0.,   0.,  40.,   0.,   0.],
        [  0.,   0.,   0.,   0., 210.,   0.],
        [ 10.,  50.,  90.,   0.,   0.,   0.]],

       [[  0.,   0.,   0., 120.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  90.,   0.],
        [110., 180., 100.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,  70.,   0.,   0.],
        [  0.,   0.,   0.,   0., 130.,   0.],
        [ 30.,  40., 130.,   0.,   0.,   0.]]])

In [180]:
n_nodes

[2, 3, 5]

In [181]:
X[0]

array([[[ 40.,   0., 150.],
        [  0., 210.,   0.],
        [  0.,   0.,   0.]],

       [[110.,   0., 390.],
        [ 10.,  90.,   0.],
        [  0.,   0.,   0.]],

       [[ 70.,   0., 200.],
        [  0., 130.,   0.],
        [  0.,   0.,   0.]]])

In [182]:
X[1]

array([[[  0.,   0.,   0.,  40.,   0.,   0.],
        [  0.,   0.,   0.,   0., 210.,   0.],
        [ 10.,  50.,  90.,   0.,   0.,   0.]],

       [[  0.,   0.,   0., 120.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  90.,   0.],
        [110., 180., 100.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,  70.,   0.,   0.],
        [  0.,   0.,   0.,   0., 130.,   0.],
        [ 30.,  40., 130.,   0.,   0.,   0.]]])

In [186]:
supplies

[[200, 300, 0], [130, 270, 200], [170, 130, 100]]

In [185]:
demands

[[10, 50, 90, 40, 210, 100],
 [110, 180, 100, 120, 90, 0],
 [30, 40, 130, 70, 130, 0]]

In [66]:
X[1]

array([[[   0.,    0.,    0.,   40.,    0.,    0.],
        [   0.,    0.,    0.,    0.,   80.,    0.],
        [ -20., -130.,  -10.,    0.,    0.,    0.]],

       [[   0.,    0.,    0.,   50.,    0.,    0.],
        [   0.,    0.,    0.,    0., -120.,    0.],
        [ 100.,  180.,  100.,    0.,    0.,    0.]],

       [[   0.,    0.,    0.,   30.,    0.,    0.],
        [   0.,    0.,    0.,    0.,  130.,    0.],
        [  30.,  -10.,   40.,    0.,    0.,    0.]]])

In [101]:
left = [[0]*3]*3
right = [[0]*5]*3
X = [np.zeros((n_nodes[i] + int(i==0), 
               n_nodes[i+1] + int(i+1==len(n_nodes)-1))) \
     for i in range(len(n_nodes)-1)]
transport(left, right, capacities[-1], demands, X, 1, 1, 2)

100

## Checkpoint

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from multiproduct import generate_dataset
n_nodes, supplies, demands, costs, capacities = generate_dataset(
                nodes=10, levels=3, total_supplies=[500,400,300], 
                total_demands=[400,600,400], transp_costs=(10,100), 
                random_state=42)

In [4]:
np.random.seed(42)
n_items = len(supplies)
chromosome = np.random.permutation(sum([n_items * sum(n_nodes[1:])]))

In [34]:
from itertools import product

def dummy_transportation(I, K, left_cap, right_cap, chrom):
    chrom = np.reshape(chrom, (I,K))
    for i, k in sorted(product(range(I), range(K)),
                       key=lambda x: chrom[x],
                       reverse=True):
        available = min(left_cap[i], right_cap[i][k])
        left_cap[i] -= available
        right_cap[i][k] -= available
    return right_cap

In [35]:
from multiproduct import transportation_stage

def transportation_tree(chromosome, n_nodes, supplies, demands, costs, capacities):
    n_items = len(demands)
    X = [np.zeros((n_items,
               n_nodes[i], 
               n_nodes[i+1])) \
         for i in range(len(n_nodes)-1)]
    # Divide chromosome in stages
    chrom_idxs = [0] + list(np.cumsum([n_items * x for x in n_nodes][1:]))
    chrom_stages = [chromosome[chrom_idxs[i]:chrom_idxs[i+1]] \
                    for i in range(len(n_nodes)-1)]
    # Dummy demand
    right_cap = [x[:-1] for x in demands]
    right_cap = dummy_transportation(n_items, n_nodes[-1],
            [x[-1] for x in supplies], right_cap, chrom_stages[-1])
    for i in range(len(n_nodes)-1):
        stage = len(n_nodes) - i - 2
        left_cap = capacities[stage] if stage > 0 else supplies
        right_cap = transportation_stage(n_items, n_nodes[stage],
                            n_nodes[stage+1], n_nodes, 
                            left_cap, right_cap, X[stage],
                            chrom_stages[stage], costs[stage])
    return X

In [36]:
X = transportation_tree(chromosome, n_nodes, supplies, demands, 
                        costs, capacities)

In [41]:
X[0]

array([[[ 40.,   0., 150.],
        [  0., 210.,   0.]],

       [[120.,   0.,  10.],
        [  0.,   0., 270.]],

       [[  0.,   0., 170.],
        [  0., 130.,   0.]]])

In [42]:
X[1]

array([[[  0.,   0.,   0.,  40.,   0.],
        [  0.,   0.,   0.,   0., 210.],
        [ 10.,  50.,  90.,   0.,   0.]],

       [[  0.,   0.,   0., 120.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [110.,  70., 100.,   0.,   0.]],

       [[  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 130.],
        [ 30.,  40., 100.,   0.,   0.]]])

In [43]:
n_nodes

[2, 3, 5]

In [47]:
[s[:-1] for s in supplies], [d[:-1] for d in demands]

([[200, 300], [130, 270], [170, 130]],
 [[10, 50, 90, 40, 210], [110, 180, 100, 120, 90], [30, 40, 130, 70, 130]])

In [49]:
sum(supplies[0]), sum(demands[0])

(500, 500)